<a href="https://colab.research.google.com/github/kynthesis/HaystackResearch/blob/main/6_Prompt_QA_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Cách xây dựng một pipeline QA generative sử dụng prompt**



# 1. Kiểm tra GPU runtime

In [1]:
%%bash

nvidia-smi

Sun Jul  2 03:56:28 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 2. Cài đặt Haystack

In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]
pip install datasets>=2.6.1

# 3. Bật chế độ logging

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# 4. Khởi tạo DocumentStore

In [4]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


# 5. Chuẩn bị các file tài liệu

In [5]:
from datasets import load_dataset

dataset = load_dataset("bilgeyucel/seven-wonders", split="train")

document_store.write_documents(dataset)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/bilgeyucel___parquet/bilgeyucel--seven-wonders-6536ac0e9c84233f/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


Updating BM25 representation...:   0%|          | 0/151 [00:00<?, ? docs/s]

# 5. Khởi tạo Retriver

In [6]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store, top_k=2)

# 6. Khởi tạo Prompt

In [7]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

lfqa_prompt = PromptTemplate(
    prompt="""Synthesize a comprehensive answer from the following text for the given question.
                             Provide a clear and concise response that summarizes the key points and information presented in the text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(model_name_or_path="google/flan-t5-large", default_prompt_template=lfqa_prompt)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


# 7. Tạo pipeline QA gồm Retriever và Prompt

In [8]:
from haystack.pipelines import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

# 8. Đặt câu hỏi cho pipeline QA

In [9]:
output = pipe.run(query="Where is Gardens of Babylon?")

Token indices sequence length is longer than the specified maximum sequence length for this model (602 > 512). Running this sequence through the model will result in indexing errors


# 9. Nhận câu trả lời từ pipeline QA

In [10]:
print(output["answers"][0].answer)

Sennacherib's palace gardens, but no evidence of the Hanging Gardens has been found at Babylon.
